# Traveling Salesman Problem using Genetic Algorithm

### importing packages to be used

In [231]:
import numpy as np
import random as random
import itertools # to check all the permutations to see how close to the optimum we got.



### A distance matrix which denotes the distance between cities/destinations


In [232]:
distance_matrix = np.array([
    [  0, 650, 150, 948, 425],
    [650,   0, 227, 625, 130],
    [150, 227,   0, 930, 120],
    [948, 625, 930,   0, 528],
    [425, 130, 120, 528,   0]
])

### Definition of functions to be used in the GA

In [233]:
# Initialize population
def create_initial_population(cities, population_size):
    population = []
    for _ in range(population_size):
        route = cities.copy()
        random.shuffle(route)
        population.append(route)
    return population


In [234]:
# Generate the initial population
# cities: list of cities, population_size: length of population
def generate_initial_population(cities, population_size):
    population = []
    for _ in range(population_size):
        route = cities.copy() #copy on route list
        random.shuffle(route) #shuffle the route list
        population.append(route) # append the list in the generated population
    return population


In [235]:
# Fitness function
def fitness(route, distance_matrix):
    total_distance = sum(distance_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
    total_distance += distance_matrix[route[-1]][route[0]]  # Return to start
    return 1 / total_distance  # Fitness is inverse of distance

In [236]:
# calculation of the distance of each route
def distance_fitness(route, distance_matrix):
    total_distance = sum(distance_matrix[route[i]][route[i+1]] for i in range(len(route)-1))
    total_distance += distance_matrix[route[-1]][route[0]]  # Return to start
    return total_distance  # Fitness is inverse of distance

In [237]:
# function used on the cumulative normalized matrix to simulte the 
# spinning roulette wheel for population selection.
def get_population_index(matrix, no):
    idx = 0
    for i in range(len(matrix)):
        if(matrix[i]>no):
            return idx
        idx+=1

In [238]:
# An order crossover implementation for the TSP.
def order_crossover(par1, par2):
    size = len(par1)

    # Choose two random crossover points
    start, end = sorted(random.sample(range(size), 2))

    # Initialize two children with None values
    child1, child2 = [None] * size, [None] * size

    # Copy the segment from Parent 1 to Child 1, and from Parent 2 to Child 2
    child1[start:end+1] = par1[start:end+1]
    child2[start:end+1] = par2[start:end+1]

    # Fill the remaining positions in Child 1 with cities from Parent 2
    pos1 = end + 1
    for city in par2:
        if city not in child1:
            if pos1 >= size:
                pos1 = 0
            child1[pos1] = city
            pos1 += 1

    # Fill the remaining positions in Child 2 with cities from Parent 1
    pos2 = end + 1
    for city in par1:
        if city not in child2:
            if pos2 >= size:
                pos2 = 0
            child2[pos2] = city
            pos2 += 1

    return child1, child2

### Initial population generation

In [252]:
print("Tackling traveling salesman problem using a genetic algorithm implementation")
epochs = 1000
mutation_rate = 0.01
population_size = 16
cities = list(range(5))
number_of_cities = len(cities)
print("cities: ",cities)

population = generate_initial_population(cities, population_size)
print("initial population created:")
for i in range(0,len(population)):
    print(population[i])



Tackling traveling salesman problem using a genetic algorithm implementation
cities:  [0, 1, 2, 3, 4]
initial population created:
[3, 0, 1, 4, 2]
[0, 1, 2, 3, 4]
[2, 0, 1, 3, 4]
[4, 3, 0, 1, 2]
[1, 3, 0, 2, 4]
[0, 4, 2, 3, 1]
[1, 3, 2, 4, 0]
[1, 0, 3, 4, 2]
[0, 1, 3, 4, 2]
[4, 1, 3, 2, 0]
[2, 0, 4, 3, 1]
[4, 3, 1, 0, 2]
[1, 2, 3, 0, 4]
[1, 3, 4, 2, 0]
[3, 4, 2, 1, 0]
[4, 2, 1, 0, 3]


### Algorithm's loop for specified number of generations/epochs.
* Evaluation of fitness function.
* normalization of fitness scores.
* selection of the new population based on the roulette wheel spin.
* application of order crossover(since the nature of the problem doesn't allow simple crossover)
* mutation step

In [254]:
for epoch in range(epochs):
    #evaluation of the objective function
    fitness_scores = [fitness(route, distance_matrix) for route in population]
    distance_scores = [distance_fitness(route, distance_matrix) for route in population]
    print("starting generation population  GENERATION["+str(epoch)+"] :")
    for i in range(0,len(population)):
        print(population[i])
    # now normalization of the function scores happens.
    normalized_function_scores = function_scores/sum(function_scores)
    new_population = [] # empty it.
    
    print("distances per route")
    for i in range(len(population)):
        print(distance_scores[i])
        
    print("normalized_f_sc")
    print(normalized_function_scores)

    cumulative_normalized_function_scores = normalized_function_scores.copy()
    for i in range(len(cumulative_normalized_function_scores)):
        for j in range(0,i):
            cumulative_normalized_function_scores[i]+=normalized_function_scores[j]
            
    #print("cumulative_n_f_s")
    #print(cumulative_normalized_function_scores)

    #selection of the new population.
    for i in range(population_size):
        random_number = random.random() #get a random number.(like spinning the roulette wheel)
        #print("random no "+str(random_number))
        ret = get_population_index(cumulative_normalized_function_scores,random_number)
        #print("assign me to "+str(ret))
        new_population.append(population[ret])


    #print("New population: ")
    #for i in range(0,len(new_population)):
        #print(new_population[i])

    # because of the nature of the traveling salesman problem crossover 
    # in its general form can not be used, so we implement order crossover.
    # we keep the rule for 0.75 3 out of 4 crossover.
    for i in range(0,round(population_size*0.75),2):
        new_population[i], new_population[i+1] = order_crossover(new_population[i], new_population[i+1])

    #print("New population after order crossover: ")
    #for i in range(0,len(new_population)):
    #    print(new_population[i])

    #Mutation step.
    # the nature of the application does not allows just a single flip.
    # so a mutation will be a 1 to 1 swap of two random cities in the mutated population.
    for i in range(population_size):
        #print("check mutation on "+str(i))
        #print(new_population[i])
        mutate = random.random() #this is a number between [0,1)
        #print("mutate value: "+str(mutate))
        #print(1-mutate)
        #print("rate of mutation "+str(mutation_rate))
        
        if( (1-mutate) < mutation_rate): #occurance of mutation
            #print("MUTATION")
            sp = random.randint(1,number_of_cities-2)
            ep = random.randint(1,number_of_cities-1)
            while(sp >= ep):
                ep = random.randint(1,number_of_cities-1)
            print(sp)
            print(ep)
            tmp = new_population[i][sp]
            new_population[i][sp] = new_population[i][ep]
            new_population[i][ep] = tmp
            #print("mutated now: ")
            #print(new_population[i])

    # set it as the new generation surviving for the next epoch.
    population = new_population


starting generation population  GENERATION[0] :
[1, 4, 2, 0, 3]
[4, 1, 3, 0, 2]
[1, 2, 4, 0, 3]
[1, 3, 4, 0, 2]
[1, 3, 4, 2, 0]
[3, 4, 1, 0, 2]
[4, 1, 2, 0, 3]
[4, 1, 3, 0, 2]
[1, 3, 4, 0, 2]
[3, 4, 1, 2, 0]
[1, 2, 4, 3, 0]
[3, 4, 1, 2, 0]
[3, 4, 1, 0, 2]
[3, 4, 1, 0, 2]
[1, 2, 4, 0, 3]
[3, 4, 1, 0, 2]
distances per route
1973
1973
2345
1955
2073
2388
1983
1973
1955
1983
2473
1983
2388
2388
2345
2388
normalized_f_sc
[0.12269939 0.14110429 0.12269939 0.12269939 0.12269939 0.12269939
 0.12269939 0.12269939]
starting generation population  GENERATION[1] :
[1, 4, 3, 0, 2]
[1, 2, 4, 0, 3]
[1, 3, 4, 0, 2]
[1, 4, 3, 0, 2]
[4, 1, 3, 0, 2]
[1, 3, 4, 0, 2]
[1, 4, 3, 0, 2]
[4, 1, 2, 0, 3]
[4, 1, 2, 0, 3]
[4, 1, 3, 0, 2]
[1, 4, 2, 0, 3]
[1, 2, 4, 0, 3]
[1, 3, 4, 0, 2]
[1, 2, 4, 0, 3]
[1, 3, 4, 0, 2]
[1, 4, 2, 0, 3]
distances per route
1983
2345
1955
1983
1973
1955
1983
1983
1983
1973
1973
2345
1955
2345
1955
1973
normalized_f_sc
[0.12269939 0.14110429 0.12269939 0.12269939 0.12269939 0.12269939
 0

In [255]:
# Generate all permutations of the cities
all_routes = list(itertools.permutations(cities))
holder = []
# Print all routes
for route in all_routes:
    distance = distance_fitness(route, distance_matrix)
    holder.append(distance)
    
print("minimum of all distances: "+str(min(holder)) + " maximum of all distances: "+str(max(holder)))

minimum of all distances: 1955 maximum of all distances: 2778
